# Coding Yahac project - 1
## Face Detector
![image](https://i.ytimg.com/vi/tpWVyJqehG4/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBvy6Os5D8npNP0A9XW_RCybrlJ-Q)  

[빵형의 개발도상국 - 얼굴인식 스노우 카메라 쉽게 따라만들기 / YOUTUBE](https://www.youtube.com/watch?v=tpWVyJqehG4)  
[소스코드 / GITHUB](https://github.com/kairess/face_detector)  
[무료 동영상 다운로드 / PEXELS](https://videos.pexels.com/search/face)

## Import Modules

In [1]:
import cv2
import dlib
import sys
import numpy as np

필요한 모듈들을 import 합니다.
* OpenCV 모듈 : 영상처리용 모듈  
  **conda**  `conda install -c menpo opencv3`  
  **pip** `pip install opencv-python` or `pip install opencv-contrib-python`  

* Dlib 모듈 : 이미지 처리용 모듈(Cmake모듈이 있어야합니다)  
  **conda** `conda intall -c menpo dlib`  
  **pip** `pip install dlib`    
  커맨드 설치안될때 : [dlib 깃허브 페이지](https://github.com/davisking/dlib) -> git clone 후 setup.py 파일 실행  
  (아무리 시도해도 안될때는)
  1. [여기서](https://pypi.org/simple/dlib/) .whl 파일을 받습니다.
  2. `conda install python=3.6` 커맨드를 입력한다.
  3. .whl 파일이 있는 경로로 가서 `pip install ~.whl` 커맨드를 입력한다.  
    
  
* Sys 모듈 : 커맨드 접근용 모듈  
  자동으로 설치되니 import만 하면됩니다.
    
* Numpy 모듈 : Array 응용 모듈  
  **conda** : `conda install -c conda-forge numpy`  
  **pip** : `pip install numpy`

## Hyperparameter

In [2]:
scaler = 0.8

OpenCV 창의 크기를 조절해줍시다  
0.8배로 할거에요

## Get Data and Model

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('dlib-models/shape_predictor_68_face_landmarks.dat')
cap = cv2.VideoCapture('face_detector/samples/Irene3.mp4')
overlay = cv2.imread('face_detector/samples/ryan_transparent.png', cv2.IMREAD_UNCHANGED)

데이터와 모델을 받아서 변수에 초기화시켜줍시다  
얼굴인식을 위해서 dlib모듈에서 얼굴인식 detector을 가져올거구요,  
얼굴 형태를 찾기위해 dlib모듈에서 prediction 모델을 가져올거에요  

실제 사용할 원본 비디오파일과 얼굴에 씌울 라이언 마스크파일도 받아와야겠죠?

## Overlay method

In [4]:
def overlay_transparent(background_img, img_to_overlay_t, x, y, overlay_size=None):
    try :
        bg_img = background_img.copy()
        # convert 3 channels to 4 channels
        if bg_img.shape[2] == 3:
            bg_img = cv2.cvtColor(bg_img, cv2.COLOR_BGR2BGRA)

        if overlay_size is not None:
            img_to_overlay_t = cv2.resize(img_to_overlay_t.copy(), overlay_size)

        b, g, r, a = cv2.split(img_to_overlay_t)

        mask = cv2.medianBlur(a, 5)

        h, w, _ = img_to_overlay_t.shape
        roi = bg_img[int(y-h/2):int(y+h/2), int(x-w/2):int(x+w/2)]

        img1_bg = cv2.bitwise_and(roi.copy(), roi.copy(), mask=cv2.bitwise_not(mask))
        img2_fg = cv2.bitwise_and(img_to_overlay_t, img_to_overlay_t, mask=mask)

        bg_img[int(y-h/2):int(y+h/2), int(x-w/2):int(x+w/2)] = cv2.add(img1_bg, img2_fg)

        # convert 4 channels to 4 channels
        bg_img = cv2.cvtColor(bg_img, cv2.COLOR_BGRA2BGR)
        return bg_img
    except Exception : return background_img

라이언 마스크를 씌우는 메소드입니다.  
어려운 코드라 빵형님도 가져오셨다고 하시네요

얼굴위에 마스크를 씌웠을때 마스크가 화면을 넘어서면 에러가 뜨더라구요  
그래서 제가 임의로 Try Exception을 넣었습니다.

## While play

여기가 좀 복잡한데... while 문으로 묶여있어서 설명하기가 좀 그렇네요

1. detector로 찾은 얼굴들(faces)을 list로 저장합니다.  
2. faces가 공백이라면 아무것도 출력하지 않습니다.  
3. 만약에 face가 있다면
  * 얼굴 형태(Face landmarks)를 찾은뒤 거기에 맞게 점을 찍어주고
  * 얼굴의 영역(roi)를 잡은다음 좌표를 생성합니다.
  * 얼굴 중앙에 라이언 마스크를 입힌다음에 얼굴크기에 맞춰서 조절해줍니다.
4. cv.imshow() 함수로 결과물을 출력합니다

In [ ]:
face_roi = []
face_sizes = []

while True:
        # read frame buffer from video
    ret, img = cap.read()
    if not ret:
        break

    # resize frame
    img = cv2.resize(img, (int(img.shape[1] * scaler), int(img.shape[0] * scaler)))
    ori = img.copy()

    # find faces
    if len(face_roi) == 0:
        faces = detector(img, 1)
    else:
        roi_img = img[face_roi[0]:face_roi[1], face_roi[2]:face_roi[3]]
        # cv2.imshow('roi', roi_img)
        faces = detector(roi_img)

    # no faces
    if len(faces) == 0:
        print('no faces!')

    # find facial landmarks
    for face in faces:
        if len(face_roi) == 0:
            dlib_shape = predictor(img, face)
            shape_2d = np.array([[p.x, p.y] for p in dlib_shape.parts()])
        else:
            dlib_shape = predictor(roi_img, face)
            shape_2d = np.array([[p.x + face_roi[2], p.y + face_roi[0]] for p in dlib_shape.parts()])

    for s in shape_2d:
        cv2.circle(img, center=tuple(s), radius=1, color=(255, 255, 255), thickness=2, lineType=cv2.LINE_AA)

        # compute face center
        center_x, center_y = np.mean(shape_2d, axis=0).astype(np.int)

        # compute face boundaries
        min_coords = np.min(shape_2d, axis=0)
        max_coords = np.max(shape_2d, axis=0)
        
        # draw min, max coords
        cv2.circle(img, center=tuple(min_coords), radius=1, color=(255, 0, 0), thickness=2, lineType=cv2.LINE_AA)
        cv2.circle(img, center=tuple(max_coords), radius=1, color=(255, 0, 0), thickness=2, lineType=cv2.LINE_AA)

        # compute face size
        face_size = max(max_coords - min_coords)
        face_sizes.append(face_size)
        if len(face_sizes) > 10:
            del face_sizes[0]
        mean_face_size = int(np.mean(face_sizes) * 1.8)

        # compute face roi
        face_roi = np.array([int(min_coords[1] - face_size / 2), int(max_coords[1] + face_size / 2), int(min_coords[0] - face_size / 2), int(max_coords[0] + face_size / 2)])
        face_roi = np.clip(face_roi, 0, 10000)
        
        # draw overlay on face
        result = overlay_transparent(ori, overlay, center_x + 8, center_y - 25, overlay_size=(mean_face_size, mean_face_size))

    # visualize
    cv2.imshow('original', ori)
    cv2.imshow('facial landmarks', img)
    cv2.imshow('result', result)
    if cv2.waitKey(1 ) == ord('q'):
        sys.exit(1)

## Output

라이언 마스크가 화면 밖으로 나올때는 적용이 안되다가  
화면 안에 들어서는 순간 잘 작동하는걸 볼 수 있다.  

![image](output.gif)